In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers, Model
import keras

In [ ]:
def training(len_input, embed_space_size, embed_action_size):
    # shape = (資料筆數, 資料維度)？
    input_space = keras.Input(shape=len_input, name='Space_input')
    input_action = keras.Input(shape=len_input, name='Action_input')

    if(embed_space_size is not None):
        space_embedding = layers.Embedding(input_dim=embed_space_size, output_dim=4, mask_zero=True, name='Space_embedding')
    else:
        space_embedding = None

    if(embed_action_size is not None):
        action_embeding = layers.Embedding(input_dim=embed_action_size, output_dim=4, mask_zero=True, name='Action_embedding')
    else:
        action_embeding = None




---

In [19]:
def sample_process():
    inputs = keras.Input(shape=(None, None, 3))
    processed = layers.RandomCrop(width=32, height=32)(inputs)
    conv = layers.Conv2D(filters=2, kernel_size=3)(processed)
    pooling = layers.GlobalAveragePooling2D()(conv)
    feature = layers.Dense(10)(pooling)

    full_model = Model(inputs, feature)
    return full_model

In [20]:
model = sample_process()
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
random_crop_2 (RandomCrop)   (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 2)         56        
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                30        
Total params: 86
Trainable params: 86
Non-trainable params: 0
_________________________________________________________________


---

In [21]:
def training_process(data_size, space_dim, action_dim):
    # shape = data sequence size
    space_input = keras.Input(shape=data_size, name='Space_input')
    action_input = keras.Input(shape=data_size, name='Action_input')

    # input_dim = each input's dimension (how many classes)
    space_embedding = layers.Embedding(input_dim=space_dim, output_dim=10, mask_zero=True, name='Space_Embedding')
    action_embedding = layers.Embedding(input_dim=action_dim, output_dim=10, mask_zero=True, name='Action_Embedding')

    # merge two embedding
    layer_concat_embedding = layers.Concatenate(name='merged_Embedding')

    layer_dense = layers.Dense(units=1, activation='sigmoid')
    
    # forward
    inputs = [space_input, action_input]
    embeded_space = space_embedding(space_input)
    embeded_action = action_embedding(action_input)
    merged_embedding = layer_concat_embedding([embeded_space, embeded_action])
    output = layer_dense(merged_embedding)

    model = Model(inputs=inputs, outputs=output)
    return model

In [22]:
model = training_process(30, 5, 5)
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Space_input (InputLayer)        [(None, 30)]         0                                            
__________________________________________________________________________________________________
Action_input (InputLayer)       [(None, 30)]         0                                            
__________________________________________________________________________________________________
Space_Embedding (Embedding)     (None, 30, 10)       50          Space_input[0][0]                
__________________________________________________________________________________________________
Action_Embedding (Embedding)    (None, 30, 10)       50          Action_input[0][0]               
____________________________________________________________________________________________